In [2]:
from sklearn.datasets import fetch_mldata
from sklearn import preprocessing
from sklearn.metrics import pairwise
import numpy as np
from scipy import sparse,spatial
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import utils as ut
import hickle as hkl

# MNIST DATASET

In [3]:
mnist = fetch_mldata('MNIST original', data_home="/Users/sasankauppu/Desktop/Data Mining CS6220/")
print mnist.data.shape
print mnist.target.shape

(70000, 784)
(70000,)


In [4]:
#mnistNorm = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True).fit(mnist.data).transform(mnist.data)
#mnistNorm = sparse.csr_matrix(mnistNorm)
mnistNormTrain = preprocessing.scale(mnist.data[:60000], with_mean=True, with_std=True, copy=True)
mnistNormTest = preprocessing.scale(mnist.data[-10000:], axis=0, with_mean=True, with_std=True, copy=True)

mnistNorm = np.concatenate((mnistNormTrain,mnistNormTest))

mnistNormTrain = None
mnistNormTest = None

/Users/sasankauppu/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
#mnistPairwise = pairwise.cosine_similarity(X=mnistNorm, Y=None, dense_output=False)
mnistPairwise = spatial.distance.pdist(mnistNorm, metric='cosine')

In [ ]:
mnistPairwise = np.around(mnistPairwise,decimals=4)

In [ ]:
%time hkl.dump(mnistPairwise, 'mnistcosinescale.hkl', mode='w', compression='gzip')

In [ ]:
'''import tables
h5file = tables.openFile('mnistcosine.h5', mode='w', title="Mnist")
root = h5file.root
h5file.createArray(root, "Mnist", mnistPairwise)

h5file.close()

#ut.saveNumpyArray(data=mnistPairwise,fname='mnistcosine.h5')
import h5py
h5f = h5py.File('mnistcosine.h5', 'w')
h5f.create_dataset('dataset1', mnistPairwise)
h5f.close()

mnistPairwise = None'''

In [ ]:
mnistPairwise = spatial.distance.pdist(mnistNorm, metric='euclidean')

In [ ]:
mnistPairwise = np.around(mnistPairwise,decimals=4)

In [ ]:
%time hkl.dump(mnistPairwise, 'mnisteuclidean.hkl', mode='w', compression='gzip')

In [ ]:
mnistPairwise = None

# NEWS GROUPS DATASET

In [3]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [4]:
newsgroupsData = np.concatenate((newsgroups_train.data,newsgroups_test.data))

In [5]:
print len(newsgroups_test.data)
print len(newsgroups_train.data)
print newsgroupsData.shape

7532
11314
(18846,)


In [ ]:
count_vect = CountVectorizer()
newsgroupsData = count_vect.fit_transform(newsgroupsData)

tfidf_transformer = TfidfTransformer(use_idf=True)
newsgroupsData = tfidf_transformer.fit_transform(newsgroupsData)

print newsgroupsData.shape

In [ ]:
ngPairwise = pairwise.cosine_similarity(X=newsgroupsData, Y=None, dense_output=False)
#ngPairwise = spatial.distance.pdist(newsgroupsData.todense(), metric='cosine')

In [ ]:
ngPairwise = np.around(ngPairwise,decimals=4)

In [ ]:
%time hkl.dump(ngPairwise, 'ngcosineidf.hkl', mode='w', compression='gzip')

In [ ]:
ngPairwise = pairwise.euclidean_distances(X=newsgroupsData, Y=None)

In [ ]:
ngPairwise = np.around(ngPairwise,decimals=4)

In [ ]:
%time hkl.dump(ngPairwise, 'ngeuclideanidf.hkl', mode='w', compression='gzip')